In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent

In [ ]:
# .env dosyası içerisinde gerekli olan keyler alınır / Required keys are retrieved from the .env file
from dotenv import load_dotenv
_ = load_dotenv()

# Key bilgisi sağlamanın diğer yolu / Another way to provide the key

# import openai
# openai.api_key = "YOUR_OPENAI_API_KEY"

In [ ]:
DATA_FOLDER = "data"

In [21]:
class AgenticRAGSystem:
    def __init__(self, data_folder: str):
        """Initialize the Agentic RAG system with LlamaIndex using OpenAI.""" # LlamaIndex ile OpenAI kullanarak Agentic RAG sistemini başlatır
        # LLM ve gömme modelleri ayarlanır / Set up LLM and embeddings
        self.llm = OpenAI(model="gpt-4o-mini")
        self.embed_model = OpenAIEmbedding()
        Settings.llm = self.llm
        Settings.embed_model = self.embed_model
        
        # Düğüm ayrıştırıcı ayarlanır / Set up node parser
        self.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
        Settings.node_parser = self.node_parser
        
        # Belgeler yüklenir ve indekslenir / Load and index documents
        self.documents = self._load_documents(data_folder)
        self.index = self._create_index()
        
        # Sorgu motoru ayarlanır / Set up query engine
        self.query_engine = self._setup_query_engine()
        
        # Ajan ayarlanır / Set up agent
        self.agent = self._setup_agent()
    
    def _load_documents(self, data_folder: str):
        """Load documents from the data folder.""" # Veri klasöründen belgeleri yükler
        if not os.path.exists(data_folder):
            raise FileNotFoundError(f"Veri klasörü '{data_folder}' bulunamadı / Data folder '{data_folder}' not found.")
        
        # Belgeleri doğru yöntemle yükler / Load documents with the correct method
        reader = SimpleDirectoryReader(input_dir=data_folder)
        return reader.load_data()
    
    def _create_index(self):
        """Create vector index from documents.""" # Belgelerden vektör indeksi oluşturur
        return VectorStoreIndex.from_documents(self.documents)
    
    def _setup_query_engine(self):
        """Set up an advanced query engine with similarity filtering.""" # Benzerlik filtreleme ile gelişmiş bir sorgu motoru ayarlar
        # Parametrelerle alıcı oluşturur / Create retriever with parameters
        retriever = VectorIndexRetriever(
            index=self.index,
            similarity_top_k=3
        )
        
        # Daha iyi sonuçlar için son işlem ekler / Add post-processing for better results
        postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)
        
        # max_tokens parametresi olmadan yanıt sentezleyici oluşturur / Create response synthesizer without max_tokens parameter
        response_synthesizer = CompactAndRefine()
        
        # Sorgu motorunu oluşturur / Build query engine
        return RetrieverQueryEngine(
            retriever=retriever,
            response_synthesizer=response_synthesizer,
            node_postprocessors=[postprocessor]
        )
    
    def _setup_agent(self):
        """Set up a ReAct agent with tools.""" # Araçlarla bir ReAct ajanı ayarlar
        # Ajan için araçlar oluşturur / Create tools for the agent
        query_tool = QueryEngineTool(
            query_engine=self.query_engine,
            metadata=ToolMetadata(
                name="document_search",
                description="Search and analyze the documents to answer questions about their content." # Belgeleri arayın ve içerikleri hakkında soruları yanıtlamak için analiz edin.
            )
        )
        
        # ReAct ajanı oluşturur / Create ReAct agent
        return ReActAgent.from_tools(
            [query_tool],
            llm=self.llm,
            verbose=True
        )
    
    def query(self, query_text: str) -> str:
        """Process a query through the agentic system.""" # Sorguyu ajansız sistem üzerinden işler
        return self.agent.query(query_text)
    
    def search_documents(self, query_text: str) -> str:
        """Direct search through the query engine (non-agentic).""" # Sorgu motoru üzerinden doğrudan arama yapar (ajansız)
        return self.query_engine.query(query_text)

In [22]:
# RAG sistemini başlat / Initialize the RAG system
rag_system = AgenticRAGSystem(
    data_folder=DATA_FOLDER
)

In [23]:
# Örnek sorgu / Example query
query = "What is the history of AI?"
print(f"Query: {query}")

# Sorgu motorundan yanıt alın / Get response from the query engine
response = rag_system.query(query)
print(f"Response: {response}")

Query: What is the history of AI?
> Running step 20691379-57bf-40e5-b768-57340299a894. Step input: What is the history of AI?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: document_search
Action Input: {'input': 'history of AI'}
Observation: The history of AI traces back to ancient myths and philosophical discussions, but it formally emerged as a field of study in the 1950s. Early models were primarily rule-based systems. Over time, advancements led to the development of deep learning and neural networks, significantly enhancing AI's capabilities. Notable milestones include IBM's Deep Blue defeating a chess champion and the creation of systems like ChatGPT that can generate human-like text.
> Running step dd5f4bb3-1807-4f25-ac55-c791fb2f662a. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The history of AI traces back to ancient myths and philosoph

In [24]:
# Örnek sorgu / Example query
query = "What is the history of AI?"
print(f"Query: {query}")

# Sorgu motorundan yanıt alın / Get response from the query engine
response = rag_system.search_documents(query)
print(f"Response: {response}")

Query: What is the history of AI?
Response: The history of AI dates back to ancient myths and early philosophical discussions, but it became a formal field of study in the 1950s. Initially, early AI models focused on rule-based systems. Over time, advancements led to the development of modern AI, which utilizes deep learning and neural networks. Significant milestones include IBM's Deep Blue defeating a chess champion and the emergence of systems like ChatGPT that can generate human-like text.
